In [1]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="bartowski/Meta-Llama-3-8B-Instruct-GGUF", filename="Meta-Llama-3-8B-Instruct-Q4_K_M.gguf", local_dir="models/llama3")

'models/llama3/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf'

In [2]:
import llama_cpp
from llama_cpp import Llama

llm = Llama(
      model_path="./models/llama3/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf",
      tokenizer=llama_cpp.llama_tokenizer.LlamaHFTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct"),
      n_gpu_layers=-1, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      n_ctx=9999, # Uncomment to increase the context window
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from ./models/llama3/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: 

In [3]:
from langchain_community.document_loaders import WebBaseLoader

def get_docs(url):
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
import time

# copy paste a url below for testing
url = "https://www.channelnewsasia.com/sport/former-germany-defender-boateng-given-court-warning-bodily-harm-case-4491596"
article = get_docs(url)

prompt_template = "summarise the following text into concise bullet points {article}"

start = time.perf_counter()
output = llm(
      prompt_template.format(article = article),
      max_tokens=1000,
      # echo=True
)
print(output)
elapsed_time = time.perf_counter() - start
print(f"llama3 inference time: {elapsed_time:.4f} seconds")


/opt/homebrew/Caskroom/miniconda/base/envs/ai_env/lib/python3.11/site-packages/llama_cpp/llama.py:1054: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(

llama_print_timings:        load time =    4116.19 ms
llama_print_timings:      sample time =      64.57 ms /   159 runs   (    0.41 ms per token,  2462.33 tokens per second)
llama_print_timings: prompt eval time =   39633.66 ms /  4429 tokens (    8.95 ms per token,   111.75 tokens per second)
llama_print_timings:        eval time =   15424.70 ms /   158 runs   (   97.62 ms per token,    10.24 tokens per second)
llama_print_timings:       total time =   61492.52 ms /  4587 tokens


{'id': 'cmpl-18dc845c-8079-4357-b113-2b8d11702599', 'object': 'text_completion', 'created': 1721401151, 'model': './models/llama3/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf', 'choices': [{'text': '}\n\nHere are the concise bullet points summarizing the text:\n\n• Former Germany defender Jerome Boateng was given a court warning and a suspended fine of €200,000 (=$217,760) for bodily harm against his former partner.\n• The 35-year-old Boateng was accused of injuring his ex-girlfriend during a fight in 2018 while on vacation.\n• He denied the charges, but the Munich court found him guilty of premeditated bodily harm.\n• The judge ordered Boateng to pay €100,000 to charity organizations and imposed a suspended fine of €5,000 per day for 40 days, which will only be paid if he commits another offense.\n• Boateng has been relieved that the trial is over and wants to focus on his family and football.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 4429, 'complet